# Intégration CoreML de RusTorch - Noyau Rust

Ce notebook démontre comment utiliser CoreML avec RusTorch.
Il s'exécute sur le noyau Rust.

## Vérifier les Dépendances et Fonctionnalités Requises

In [ ]:
// Utilisation de base de RusTorch
extern crate rustorch;

use rustorch::tensor::Tensor;
use rustorch::gpu::DeviceType;

println!("Version de RusTorch : {}", env!("CARGO_PKG_VERSION"));
println!("Version de Rust : {}", env!("RUSTC_VERSION"));

## Vérifier la Disponibilité de CoreML

Vérifier si CoreML est disponible sur le système actuel.

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::DeviceManager;
    
    let coreml_available = DeviceManager::is_coreml_available();
    println!("CoreML disponible : {}", coreml_available);
    
    if coreml_available {
        println!("🎉 CoreML est disponible !");
        println!("Plateforme : macOS");
        
        // Afficher les informations du périphérique
        use rustorch::gpu::coreml::device_cache::DeviceCache;
        let cache = DeviceCache::global();
        cache.warmup();
        
        let stats = cache.get_stats();
        println!("Statistiques du cache : {:?}", stats);
    } else {
        println!("⚠️ CoreML n'est pas disponible");
        println!("Veuillez utiliser le CPU ou d'autres backends GPU");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("❌ Les fonctionnalités CoreML ne sont pas activées");
    println!("Veuillez construire avec --features coreml");
}

## Opérations de Tenseurs de Base

In [ ]:
// Créer des tenseurs de base
let a = Tensor::zeros(&[2, 3]);
let b = Tensor::ones(&[3, 2]);

println!("Forme du tenseur A : {:?}", a.shape());
println!("Forme du tenseur B : {:?}", b.shape());

// Multiplication de matrices de base
let result = a.matmul(&b);
println!("Forme du résultat : {:?}", result.shape());
println!("Opérations de tenseurs de base terminées");

## Opérations avec Périphérique CoreML

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::gpu::coreml::{CoreMLDevice, CoreMLBackend};
    use rustorch::backends::BackendConfig;
    
    // Essayer de créer un périphérique CoreML
    match CoreMLDevice::new(0) {
        Ok(device) => {
            println!("🖥️ Périphérique CoreML créé avec succès");
            println!("ID du périphérique : {}", device.id());
            println!("Disponible : {}", device.is_available());
            println!("Limite de mémoire : {} MB", device.memory_limit() / (1024 * 1024));
            
            // Créer la configuration du backend
            let config = BackendConfig::new()
                .with_caching(true)
                .with_max_cache_size(200)
                .with_profiling(true)
                .with_auto_fallback(true);
            
            println!("⚙️ Configuration du backend : {:?}", config);
            
            // Créer le backend CoreML
            match CoreMLBackend::new(device, config) {
                Ok(backend) => {
                    println!("🚀 Backend CoreML initialisé");
                    
                    // Obtenir les statistiques
                    let stats = backend.get_statistics();
                    println!("📊 Statistiques du backend :");
                    println!("  Opérations totales : {}", stats.total_operations);
                    println!("  Succès du cache : {}", stats.cache_hits);
                    println!("  Échecs du cache : {}", stats.cache_misses);
                    println!("  Opérations de basculement : {}", stats.fallback_operations);
                    
                    // Créer des tenseurs sur CoreML
                    let tensor_a = Tensor::randn(&[64, 64]).to_device(&backend);
                    let tensor_b = Tensor::randn(&[64, 64]).to_device(&backend);
                    
                    println!("📐 Tenseurs créés sur le périphérique CoreML");
                    
                    // Opération de multiplication de matrices
                    let start = std::time::Instant::now();
                    let result = tensor_a.matmul(&tensor_b);
                    let duration = start.elapsed();
                    
                    println!("✅ Multiplication de matrices terminée");
                    println!("⏱️ Temps d'exécution : {:?}", duration);
                    println!("🎯 Forme du résultat : {:?}", result.shape());
                    
                    // Nettoyer le cache
                    backend.cleanup_cache();
                    println!("🧹 Cache nettoyé");
                }
                Err(e) => println!("❌ Erreur lors de la création du backend CoreML : {:?}", e),
            }
        }
        Err(e) => {
            println!("❌ Erreur lors de la création du périphérique CoreML : {:?}", e);
            println!("CoreML peut ne pas être disponible sur ce système");
        }
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Saut des opérations CoreML - fonctionnalités non activées");
}

## Comparaison de Performance

In [ ]:
use std::time::Instant;

fn benchmark_operations() {
    let sizes = vec![(64, 64), (128, 128), (256, 256), (512, 512)];
    
    println!("🏁 Évaluation comparative des opérations :");
    println!("Taille\t\tCPU (ms)\tPériphérique Préféré");
    println!("-" * 48);
    
    for (rows, cols) in sizes {
        // Créer des tenseurs sur CPU
        let a = Tensor::randn(&[rows, cols]);
        let b = Tensor::randn(&[cols, rows]);
        
        // Mesurer le temps CPU
        let start = Instant::now();
        let _result = a.matmul(&b);
        let cpu_duration = start.elapsed();
        
        // Déterminer le périphérique préféré
        let preferred_device = if rows * cols < 1000 {
            "CPU"
        } else if rows * cols < 10000 {
            "GPU Metal"
        } else {
            "CoreML"
        };
        
        println!("{}x{}\t\t{:.2}\t\t{}", 
                rows, cols, 
                cpu_duration.as_millis() as f64, 
                preferred_device);
    }
}

benchmark_operations();
println!("\n📝 Note : La sélection de périphérique est basée sur la taille du tenseur et la disponibilité");

## Sélection Intelligente de Périphérique

In [ ]:
#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    use rustorch::backends::{DeviceManager, DeviceSelector};
    
    fn demonstrate_device_selection() {
        println!("🎯 Démonstration de la sélection intelligente de périphérique :");
        
        let operations = vec![
            ("Multiplication petite", vec![16, 16], "CPU"),
            ("Convolution 2D", vec![32, 3, 224, 224], "CoreML"),
            ("Transformation de matrice", vec![128, 128], "GPU Metal"),
            ("Opération de lot importante", vec![512, 512], "CoreML"),
            ("Calcul vectoriel", vec![1000], "CPU"),
        ];
        
        for (name, shape, preferred) in operations {
            println!("  {:<25} {:?} -> {}", name, shape, preferred);
            
            // Simuler la sélection basée sur les règles
            let tensor_size: usize = shape.iter().product();
            let selected_device = match tensor_size {
                size if size < 1000 => DeviceType::Cpu,
                size if size < 50000 => DeviceType::MetalGpu,
                _ => {
                    if DeviceManager::is_coreml_available() {
                        DeviceType::CoreML
                    } else {
                        DeviceType::MetalGpu
                    }
                }
            };
            
            println!("    -> Périphérique sélectionné : {:?}", selected_device);
        }
        
        println!("\n📝 Logique de sélection :");
        println!("  • < 1K éléments : CPU (surcharge minimale)");
        println!("  • 1K-50K éléments : GPU Metal (équilibré)");
        println!("  • > 50K éléments : CoreML (optimisé) ou GPU Metal (basculement)");
    }
    
    demonstrate_device_selection();
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Démonstration de sélection de périphérique sautée - fonctionnalités CoreML non disponibles");
}

## Exemple Avancé : Couche de Réseau Neuronal

In [ ]:
fn simulate_neural_layer() {
    println!("🧠 Simulation de couche de réseau neuronal :");
    
    // Configuration de la couche
    let batch_size = 32;
    let input_dim = 784;   // 28x28 MNIST
    let hidden_dim = 256;
    let output_dim = 10;   // 10 classes
    
    println!("📊 Configuration :");
    println!("  Taille du lot : {}", batch_size);
    println!("  Dimension d'entrée : {}", input_dim);
    println!("  Dimension cachée : {}", hidden_dim);
    println!("  Dimension de sortie : {}", output_dim);
    
    // Créer des tenseurs
    let input = Tensor::randn(&[batch_size, input_dim]);
    let weight1 = Tensor::randn(&[input_dim, hidden_dim]);
    let weight2 = Tensor::randn(&[hidden_dim, output_dim]);
    
    println!("\n🔄 Passe avant :");
    
    // Passe avant simulée
    let start = Instant::now();
    
    // Couche 1 : entrée -> cachée
    let hidden = input.matmul(&weight1);
    println!("  ✅ Entrée -> Cachée : {:?}", hidden.shape());
    
    // Fonction d'activation ReLU (simulée)
    let activated = hidden.relu();
    println!("  ✅ Activation ReLU appliquée");
    
    // Couche 2 : cachée -> sortie
    let output = activated.matmul(&weight2);
    println!("  ✅ Cachée -> Sortie : {:?}", output.shape());
    
    let total_time = start.elapsed();
    
    println!("\n⏱️ Temps total de la passe avant : {:?}", total_time);
    println!("🚀 Performance estimée : {:.0} échantillons/seconde", 
             (batch_size as f64) / total_time.as_secs_f64());
    
    println!("\n📝 Dans une implémentation réelle :");
    println!("  • Les grandes matrices utiliseraient CoreML");
    println!("  • Les activations utiliseraient le GPU Metal");
    println!("  • Les petites opérations resteraient sur CPU");
}

simulate_neural_layer();

## Gestion d'Erreurs et Basculement

In [ ]:
fn demonstrate_fallback_behavior() {
    println!("🔄 Démonstration du comportement de basculement :");
    
    // Simuler une opération qui pourrait échouer sur CoreML
    let complex_tensor = Tensor::randn(&[100, 100]);
    
    println!("🎯 Tentative d'opération CoreML...");
    
    // Dans l'implémentation réelle, ce serait :
    // match tensor.to_coreml() {
    //     Ok(coreml_tensor) => { /* utiliser CoreML */ },
    //     Err(_) => { /* basculer vers Metal/CPU */ }
    // }
    
    let use_coreml = false; // Simuler l'échec CoreML
    
    if use_coreml {
        println!("✅ Opération CoreML réussie");
    } else {
        println!("⚠️ CoreML non disponible, utilisation du basculement");
        
        // Basculer vers le GPU Metal
        let start = Instant::now();
        let result = complex_tensor.matmul(&complex_tensor);
        let fallback_time = start.elapsed();
        
        println!("✅ Opération de basculement terminée");
        println!("⏱️ Temps de basculement : {:?}", fallback_time);
        println!("📐 Forme du résultat : {:?}", result.shape());
    }
    
    println!("\n📝 Stratégie de basculement :");
    println!("  1. Essayer CoreML (meilleures performances)");
    println!("  2. Basculer vers GPU Metal (bonne compatibilité)");
    println!("  3. Basculement final vers CPU (compatibilité maximale)");
}

demonstrate_fallback_behavior();

## Résumé et Prochaines Étapes

In [ ]:
println!("📋 Résumé de l'Intégration CoreML RusTorch (Noyau Rust) :");
println!();
println!("✅ Fonctionnalités démontrées :");
println!("  • Vérification de la disponibilité CoreML");
println!("  • Création et gestion de périphériques");
println!("  • Configuration du backend");
println!("  • Opérations de tenseurs de base");
println!("  • Évaluation comparative des performances");
println!("  • Sélection intelligente de périphérique");
println!("  • Comportement de basculement");
println!();
println!("🚧 Zone de développement :");
println!("  • Implémentation complète des opérations CoreML");
println!("  • Optimisation du transfert de mémoire");
println!("  • Support étendu pour les types de tenseurs");
println!("  • Profilage détaillé des performances");
println!("  • Intégration avec les pipelines ML");
println!();
println!("🎯 Prochaines étapes recommandées :");
println!("  1. Tester avec des modèles CoreML pré-entraînés");
println!("  2. Évaluer comparativement avec d'autres backends");
println!("  3. Optimiser pour des cas d'usage spécifiques");
println!("  4. Déployer dans des applications de production");
println!();

#[cfg(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback"))]
{
    if rustorch::backends::DeviceManager::is_coreml_available() {
        println!("🎉 Toutes les fonctionnalités CoreML sont disponibles pour les tests !");
    } else {
        println!("⚠️ CoreML est activé mais non disponible sur ce système");
    }
}

#[cfg(not(any(feature = "coreml", feature = "coreml-hybrid", feature = "coreml-fallback")))]
{
    println!("⚠️ Construisez avec les fonctionnalités CoreML pour une fonctionnalité complète");
}

println!("\n🚀 Prêt pour le développement avancé CoreML avec RusTorch !");